# Contents

[Overview](#Overview)

[Tracing code](#Tracing-code)
  * [Syntax](#Syntax)
  * [Implementation](#Implementation-of-@trace)

[Manipulating traces](#Manipulating-traces)
  * [Custom text/HTML](#Custom-text/HTML)
  * [Map, filter, collect](#Map,-filter,-collect)

[Working with mutable state](#Working-with-mutable-state)

[Debugging with traces](#Debugging-with-traces)
  * [Debugging test failures](#Debugging-test-failures)

[Profiling](#Profiling)

# Overview

TraceCalls.jl is a functional tracing package, for debugging and exploring Julia code. It records and displays a tree of function calls. For example, here is how [Calculus.jl](https://github.com/johnmyleswhite/Calculus.jl) computes the second-derivative of `sin(x) + sqrt(x)`:

In [1]:
using Calculus, TraceCalls

@traceable f(x) = sin(x) + sqrt(x)
trace_derivative = @trace Calculus second_derivative(f, 1.0)

INFO: Recompiling stale cache file /Users/cedric/.julia/lib/v0.6/TraceCalls.ji for module TraceCalls.


TraceCalls.Trace(#1, (), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.second_derivative, (f, 1.0), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :scalar, :central), (), TraceCalls.Trace[], Calculus.#27)], Calculus.#27), TraceCalls.Trace(Calculus.finite_difference_hessian, (f, Calculus.#27, 1.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (Calculus.#27, 1.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (f, 1.00001, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.00001,), (), TraceCalls.Trace[], 1.84148), TraceCalls.Trace(f, (1.0,), (), TraceCalls.Trace[], 1.84147)], 1.0403), TraceCalls.Trace(Calculus.finite_difference, (f, 0.999994, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.0,), (), TraceCalls.Trace[], 1.84147), TraceCalls.Trace(f, (0.999988,), (), TraceCalls.Trace[], 1.84146)], 1.04031)], -1.09147)], -1.09147)], -1.09147)], -1.0914708926553454)

The output of `@trace` is a `Trace` object --- an explorable tree-like datastructure:

In [2]:
trace_derivative[1][1]          # get the first call of the first call.

TraceCalls.Trace(Calculus.derivative, (f, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :scalar, :central), (), TraceCalls.Trace[], Calculus.#27)], Calculus.#27)

In [3]:
trace_derivative[1][1].args[2]  # get its second argument

:central

It can work as a [more-informative stack-trace](#Debugging-with-traces) (which contains _values_ rather than just types - see `@stacktrace`):

In [4]:
@trace Calculus second_derivative(f, 0.0)

TraceCalls.Trace(#3, (), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.second_derivative, (f, 0.0), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :scalar, :central), (), TraceCalls.Trace[], Calculus.#65)], Calculus.#65), TraceCalls.Trace(Calculus.finite_difference_hessian, (f, Calculus.#65, 0.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (Calculus.#65, 0.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (f, 6.05545e-6, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.21109e-5,), (), TraceCalls.Trace[], 0.00349218), TraceCalls.Trace(f, (0.0,), (), TraceCalls.Trace[], 0.0)], 288.35), TraceCalls.Trace(Calculus.finite_difference, (f, -6.05545e-6, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (0.0,), (), TraceCalls.Trace[], 0.0), TraceCalls.Trace(f, (-1.21109e-5,), (), TraceCalls.Trace[], DomainError())], DomainError())], DomainError())], DomainError())], DomainError())], DomainError())

And finally, since full call data is recorded, we can rerun every part of the trace for [profiling](#Profiling), debugging, or testing.

In [26]:
greenred(map(:@allocated, trace_derivative))    # compute how many bytes were allocated in each function call

TraceCalls.Trace(#1, (), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.second_derivative, (f, 1.0), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :scalar, :central), (), TraceCalls.Trace[], TraceCalls.FontColor("ff0000", 208))], TraceCalls.FontColor("ff0000", 208)), TraceCalls.Trace(Calculus.finite_difference_hessian, (f, Calculus.#27, 1.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (Calculus.#27, 1.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (f, 1.00001, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.00001,), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0)), TraceCalls.Trace(f, (1.0,), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0))], TraceCalls.FontColor("00ff00", 0)), TraceCalls.Trace(Calculus.finite_difference, (f, 0.999994, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.0,), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0)), TraceCalls.Trace(f, (0.999988,), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0))], TraceCalls.FontColor("00ff00", 0))], TraceCalls.FontColor("ff0000", 16))], TraceCalls.FontColor("ff0000", 16))], TraceCalls.FontColor("ff0000", 256))], TraceCalls.FontColor("ff0000", 256))

# Tracing code

The best way to trace your own codebase is to annotate key functions with `@traceable`.

In [6]:
@traceable function my_factorial(n)
    if n==0
        return 1
    else
        return n * my_factorial(n-1)
    end
end

# How many ways there are of selecting k objects amongst n. See https://en.wikipedia.org/wiki/Combination
combination(k, n) = my_factorial(n) / (my_factorial(n-k) * my_factorial(k))

@trace combination(2, 3)

TraceCalls.Trace(#17, (), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (3,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (2,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (1,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (0,), (), TraceCalls.Trace[], 1)], 1)], 2)], 6), TraceCalls.Trace(my_factorial, (1,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (0,), (), TraceCalls.Trace[], 1)], 1), TraceCalls.Trace(my_factorial, (2,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (1,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (0,), (), TraceCalls.Trace[], 1)], 1)], 2)], 3.0)

In the example above, `my_factorial` is traced (each call to it will be recorded when `@trace` is used), but `combination` isn't. Crucially, the `@traceable` macro merely remembers the function definition, it does not modify it in any way, which means that **@traceable has zero impact on the performance of your code**. Sprinkle it liberally!

When it's not practical to add `@traceable` annotations, you can trace entire modules or specific functions by passing them to the `@trace` macro:

In [7]:
using DataStructures
to_trace = (DataStructures.nextreme, compare)
@trace to_trace nlargest(3, [0,21,-12,68,-25,14])  # returns the 3 largest values in the vector

TraceCalls.Trace(#19, (), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.nextreme, (DataStructures.LessThan(), 3, [0, 21, -12, 68, -25, 14]), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 21, 0), (), TraceCalls.Trace[], false), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), -12, 0), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), -12, 68), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 21, 0), (), TraceCalls.Trace[], false), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 68, 0), (), TraceCalls.Trace[], false), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 0, -25), (), TraceCalls.Trace[], false), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 0, 14), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 21, 68), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 14, 21), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 68, 21), (), TraceCalls.Trace[], false)], [68, 21, 14])], [68, 21, 14])

`@trace` attempts to find the source of every method of the traced functions, and emits a warning (see above) when it is missing some. When tracing modules, `@trace` has to parse their source code looking for function definitions (using [Revise.jl](https://github.com/timholy/Revise.jl)). That is an inexact science; many functions will not be traced. In particular, `@trace` and `@traceable` ignore:

 - Inner constructors
 - Functions that are defined by `eval`
 - Function definitions inside a macro (eg. `@inline foo(x) = x+2`), unless `@traceable` is part of its function definition.
 - Callable objects (eg. `(func::Functor)(x) = ...`)

#### Syntax

The full syntax of `@trace` is:

    @trace (object1, object2, ...) code_to_execute
    
where each `object` can be a function, a module, or a filename (that was already interactively `include`d). It will trace the methods defined in the objects, _and all methods that were defined with `@traceable`_. The shorthand `@trace code_to_trace` is equivalent to `@trace () code_to_trace`.

#### Implementation of @trace

`@trace some_fn foo(10)` performs these operations:

1. Replace the definition(s) of `some_fn` with a tracing version (using `eval`)
2. Run `foo(10)`
3. Restore the original definition(s) of `some_fn` (using `eval`)

The downside of this scheme is that `@trace` can cause significant JIT compiling time. Tracing large modules can be slow the first time around, but caching is used to avoid repeated computations.

# Manipulating traces

Consider computing a [random walk](https://en.wikipedia.org/wiki/Random_walk) on a small graph, using [LightGraphs.jl](http://juliagraphs.github.io/LightGraphs.jl/latest/)

In [8]:
using LightGraphs

graph = Graph(3)                                      # build an undirected graph with three connected vertices
add_edge!(graph, 1, 2); add_edge!(graph, 2, 3) 

trace_walk = @trace LightGraphs randomwalk(graph, 2, 5)

TraceCalls.Trace(#21, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.randomwalk, ({3, 2} undirected simple Int64 graph, 2, 5), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.vertices, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.nv, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Array{Int64,1}[[2], [1, 3], [2]])], 3)], 1:3), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], [2])], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 3), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 3), (), TraceCalls.Trace[], [2])], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3])], [2, 1, 2, 3, 2])], [2, 1, 2, 3, 2])

The trace can be indexed:

In [9]:
trace_walk[1][3]    # can also be written graph_trace[1,3]

TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3])

Called:

In [10]:
trace_walk[1,3]()   # call `LightGraphs.out_neighbors({2, 1} directed simple Int64 graph, 2)`

2-element Array{Int64,1}:
 1
 3

Or pruned (very useful for exploring large traces):

In [11]:
prune(trace_walk, 
    2,  # maximum depth
    4)  # maximum length of each trace (eg. if foo() calls bar() 100 times)

TraceCalls.Trace(#21, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.randomwalk, ({3, 2} undirected simple Int64 graph, 2, 5), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.vertices, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], 1:3), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], [2])], [2, 1, 2, 3, 2])], [2, 1, 2, 3, 2])

#### Custom text/HTML

To display each argument and return value, `TraceCalls.show_val(io::IO, mime, x)` is called. It defaults to `show(io, x)`, but can be customized, either to highlight certain values, or to shorten objects that are not important for the task at hand.

In [12]:
TraceCalls.show_val(io::IO, ::MIME"text/html", v::Vector{Int}) = write(io, string("[", join([x==2 ? "<font color=red>2</font>" : x for x in v], ","), "]"))
TraceCalls.show_val(io::IO, ::MIME"text/html", ::Graph) = write(io, "AnyOldGraph")
trace_walk

TraceCalls.Trace(#21, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.randomwalk, ({3, 2} undirected simple Int64 graph, 2, 5), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.vertices, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.nv, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Array{Int64,1}[[2], [1, 3], [2]])], 3)], 1:3), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], [2])], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 3), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 3), (), TraceCalls.Trace[], [2])], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3])], [2, 1, 2, 3, 2])], [2, 1, 2, 3, 2])

See also `?TraceCalls.show_call` and `?TraceCalls.show_return_val`.

#### Map, filter, collect

Each function call in a trace is represented by an instance of the `Trace` structure:

```julia
struct Trace
    func::Function        # the function called
    args::Tuple           # the positional arguments
    kwargs::Tuple         # the keyword arguments
    called::Vector{Trace} # the functions called within the execution of this function call 
    value                 # This is the return value of the func(args...; kwargs...) call, but it's also where
                          # the result of `map(f, ::Trace)` will be stored.
end
```

Filtering works as you might expect:

In [13]:
# Get rid of the `fadj` calls
filter(trace->trace.func != LightGraphs.SimpleGraphs.fadj, trace_walk)

TraceCalls.Trace(#21, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.randomwalk, ({3, 2} undirected simple Int64 graph, 2, 5), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.vertices, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.nv, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64)], 3)], 1:3), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 3), (), TraceCalls.Trace[], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [2, 1, 2, 3, 2])], [2, 1, 2, 3, 2])

While `filter` is useful to cut out uninteresting intermediate functions (it keeps the filtered out Trace's children), `filter_cutting(f, trace)` will remove all descendents of the traces for which `f(trace)` is false.

`map(f, trace)` applies `f` (whether a function or quoted macro such as `:@which`) to each `Trace`, and stores the result in `Trace`'s `value` field. It's most useful for [Profiling](#Profiling).

In [14]:
# Take the second argument of every call to LightGraphs.out_neighbors
map(trace->trace.func == LightGraphs.out_neighbors ? trace.args[2] : nothing,
    filter(trace->trace.func == LightGraphs.out_neighbors, trace_walk))

TraceCalls.Trace(#21, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], 2), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], 1), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], 2), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 3), (), TraceCalls.Trace[], 3), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], 2)], nothing)

`collect(trace)` returns a `Vector` of all `Trace` objects in `trace`. 

# Working with mutable state

Because `@trace` stores the function call's arguments without copying them, tracing functions that modify their arguments can yield surprising results. Consider generating a vector of `n` 5s using `push!`:

In [16]:
@traceable push5!(vec::Vector) = push!(vec, 5)
@traceable function many_5s(n)
    vec = Int[]
    for i in 1:n
        push5!(vec)
    end
    return vec
end

@trace many_5s(3)

TraceCalls.Trace(#31, (), (), TraceCalls.Trace[TraceCalls.Trace(many_5s, (3,), (), TraceCalls.Trace[TraceCalls.Trace(push5!, ([5, 5, 5],), (), TraceCalls.Trace[], [5, 5, 5]), TraceCalls.Trace(push5!, ([5, 5, 5],), (), TraceCalls.Trace[], [5, 5, 5]), TraceCalls.Trace(push5!, ([5, 5, 5],), (), TraceCalls.Trace[], [5, 5, 5])], [5, 5, 5])], [5, 5, 5])

When `push5!` was first called, `vec` was empty, but this trace makes it look like it already had three 5s in it. This is because all vectors in that trace are [the same object](http://www.johnmyleswhite.com/notebook/2014/09/06/values-vs-bindings-the-map-is-not-the-territory/). As long as you keep this behaviour in mind, it is not necessarily a problem. But if you do care, here are a few solutions. The simplest is to take the mutating functions out of the trace. This isn't perfect, but it's sometimes a good option for profiling.

In [17]:
tr = @trace many_5s(3)
filter(!is_mutating, tr)    # filter out every function that ends with a ! (see https://docs.julialang.org/en/stable/manual/style-guide/#Append-!-to-names-of-functions-that-modify-their-arguments-1)

TraceCalls.Trace(#33, (), (), TraceCalls.Trace[TraceCalls.Trace(many_5s, (3,), (), TraceCalls.Trace[], [5, 5, 5])], [5, 5, 5])

Alternatively, you can tell TraceCalls to make a copy of each `Vector` argument when storing it:

In [18]:
TraceCalls.store(x::Vector) = copy(x)
@trace many_5s(3)

TraceCalls.Trace(#35, (), (), TraceCalls.Trace[TraceCalls.Trace(many_5s, (3,), (), TraceCalls.Trace[TraceCalls.Trace(push5!, (Int64[],), (), TraceCalls.Trace[], [5]), TraceCalls.Trace(push5!, ([5],), (), TraceCalls.Trace[], [5, 5]), TraceCalls.Trace(push5!, ([5, 5],), (), TraceCalls.Trace[], [5, 5, 5])], [5, 5, 5])], [5, 5, 5])

The most drastic solution is to have TraceCalls store the HTML representation of every object:

```julia
TraceCalls.store(x) = REPR(x)
```

This essentially turns TraceCalls.jl into a traditional, non-inspectable tracing package, but it guarantees that each value is shown as it was when the call was made.

# Debugging with traces

The starting point for debugging exceptions is `@stacktrace`. It's like `@trace`, but uses `filter` to keep only the part of the trace involved in the exception.

In [19]:
using Optim, TraceCalls
@traceable logplus10(x) = log(x[1]+10)
TraceCalls.store(v::Vector) = copy(v)

# Minimize the function x -> log(x[1]+10) starting at x = 0
strace = @stacktrace (Optim, Calculus) optimize(logplus10, [0.0], BFGS())

TraceCalls.Trace(#37, (), (), TraceCalls.Trace[TraceCalls.Trace(Optim.optimize, (logplus10, [0.0], Optim.BFGS{LineSearches.#hagerzhang!,Optim.##161#165}(LineSearches.hagerzhang!, Optim.#161, true), Optim.Options{Void}(1.0e-32, 1.0e-32, 1.0e-8, false, 1000, false, false, false, false, 1, nothing, NaN)), (), TraceCalls.Trace[TraceCalls.Trace(Optim.optimize, (Optim.OnceDifferentiable(logplus10, Optim.g!, Optim.fg!), [0.0], Optim.BFGS{LineSearches.#hagerzhang!,Optim.##161#165}(LineSearches.hagerzhang!, Optim.#161, true), Optim.Options{Void}(1.0e-32, 1.0e-32, 1.0e-8, false, 1000, false, false, false, false, 1, nothing, NaN)), (), TraceCalls.Trace[TraceCalls.Trace(Optim.update_state!, (Optim.OnceDifferentiable(logplus10, Optim.g!, Optim.fg!), Optim.BFGSState{Float64,1,Array{Float64,1}}("BFGS", 1, [0.0], 2.30259, 1, 1, 0, [0.0], [0.1], [0.1], NaN, [0.0], [0.0], [0.0], [1.0], [-0.1], [-12.4999], [0.133333], 1.0, false, LineSearches.LineSearchResults{Float64}([0.0, 1.0, 5.0, 25.0], [2.30259, 2.29253, 2.25129, 2.0149], [-0.01, -0.010101, -0.0105263, -0.0133333], 0)), Optim.BFGS{LineSearches.#hagerzhang!,Optim.##161#165}(LineSearches.hagerzhang!, Optim.#161, true)), (), TraceCalls.Trace[TraceCalls.Trace(Optim.perform_linesearch!, (Optim.BFGSState{Float64,1,Array{Float64,1}}("BFGS", 1, [0.0], 2.30259, 1, 1, 0, [0.0], [0.1], [0.1], NaN, [0.0], [0.0], [0.0], [1.0], [-0.1], [-12.4999], [0.133333], 1.0, false, LineSearches.LineSearchResults{Float64}([0.0, 1.0, 5.0, 25.0], [2.30259, 2.29253, 2.25129, 2.0149], [-0.01, -0.010101, -0.0105263, -0.0133333], 0)), Optim.BFGS{LineSearches.#hagerzhang!,Optim.##161#165}(LineSearches.hagerzhang!, Optim.#161, true), Optim.OnceDifferentiable(logplus10, Optim.g!, Optim.fg!)), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-12.5], [0.133333], :central), (), TraceCalls.Trace[TraceCalls.Trace(logplus10, ([-12.4999],), (), TraceCalls.Trace[], DomainError())], DomainError())], DomainError())], DomainError())], DomainError())], DomainError())], DomainError())

Then we can zoom in on the most interesting part of the stack-trace:

In [20]:
strace[1,1,1,1]

TraceCalls.Trace(Optim.perform_linesearch!, (Optim.BFGSState{Float64,1,Array{Float64,1}}("BFGS", 1, [0.0], 2.302585092994046, 1, 1, 0, [0.0], [0.1], [0.1], NaN, [0.0], [0.0], [0.0], [1.0], [-0.1], [-12.4999], [0.133333], 1.0, false, LineSearches.LineSearchResults{Float64}([0.0, 1.0, 5.0, 25.0], [2.30259, 2.29253, 2.25129, 2.0149], [-0.01, -0.010101, -0.0105263, -0.0133333], 0)), Optim.BFGS{LineSearches.#hagerzhang!,Optim.##161#165}(LineSearches.hagerzhang!, Optim.#161, true), Optim.OnceDifferentiable(logplus10, Optim.g!, Optim.fg!)), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-12.5], [0.133333], :central), (), TraceCalls.Trace[TraceCalls.Trace(logplus10, ([-12.4999],), (), TraceCalls.Trace[], DomainError())], DomainError())], DomainError())

Calling a `Trace` object repeats its computation. By tracing `strace[1,1,1,1]()`, we get the full trace for this part of the computation:

In [21]:
linesearch_trace = @trace (Optim, Calculus) strace[1,1,1,1]()

TraceCalls.Trace(#39, (), (), TraceCalls.Trace[TraceCalls.Trace(Optim.perform_linesearch!, (Optim.BFGSState{Float64,1,Array{Float64,1}}("BFGS", 1, [0.0], 2.30259, 1, 1, 0, [0.0], [0.1], [0.1], NaN, [0.0], [0.0], [0.0], [1.0], [-0.1], [-12.4999], [0.133333], 1.0, false, LineSearches.LineSearchResults{Float64}([0.0, 1.0, 5.0, 25.0], [2.30259, 2.29253, 2.25129, 2.0149], [-0.01, -0.010101, -0.0105263, -0.0133333], 0)), Optim.BFGS{LineSearches.#hagerzhang!,Optim.##161#165}(LineSearches.hagerzhang!, Optim.#161, true), Optim.OnceDifferentiable(logplus10, Optim.g!, Optim.fg!)), (), TraceCalls.Trace[TraceCalls.Trace(Optim.checked_dphi0!, (Optim.BFGSState{Float64,1,Array{Float64,1}}("BFGS", 1, [0.0], 2.30259, 1, 1, 0, [0.0], [0.1], [0.1], NaN, [0.0], [0.0], [0.0], [1.0], [-0.1], [-12.4999], [0.133333], 1.0, false, LineSearches.LineSearchResults{Float64}([0.0, 1.0, 5.0, 25.0], [2.30259, 2.29253, 2.25129, 2.0149], [-0.01, -0.010101, -0.0105263, -0.0133333], 0)), Optim.BFGS{LineSearches.#hagerzhang!,Optim.##161#165}(LineSearches.hagerzhang!, Optim.#161, true)), (), TraceCalls.Trace[], -0.01), TraceCalls.Trace(Optim.alphaguess!, (Optim.BFGSState{Float64,1,Array{Float64,1}}("BFGS", 1, [0.0], 2.30259, 1, 1, 0, [0.0], [0.1], [0.1], NaN, [0.0], [0.0], [0.0], [1.0], [-0.1], [-12.4999], [0.133333], 1.0, false, LineSearches.LineSearchResults{Float64}([0.0, 1.0, 5.0, 25.0], [2.30259, 2.29253, 2.25129, 2.0149], [-0.01, -0.010101, -0.0105263, -0.0133333], 0)), Optim.BFGS{LineSearches.#hagerzhang!,Optim.##161#165}(LineSearches.hagerzhang!, Optim.#161, true), -0.01, Optim.OnceDifferentiable(logplus10, Optim.g!, Optim.fg!)), (), TraceCalls.Trace[], 1.0), TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-0.1], [0.133333], :central), (), TraceCalls.Trace[TraceCalls.Trace(logplus10, ([-0.0999939],), (), TraceCalls.Trace[], 2.29254), TraceCalls.Trace(logplus10, ([-0.100006],), (), TraceCalls.Trace[], 2.29253)], nothing), TraceCalls.Trace(logplus10, ([-0.1],), (), TraceCalls.Trace[], 2.29253), TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-0.5], [0.10101], :central), (), TraceCalls.Trace[TraceCalls.Trace(logplus10, ([-0.499994],), (), TraceCalls.Trace[], 2.25129), TraceCalls.Trace(logplus10, ([-0.500006],), (), TraceCalls.Trace[], 2.25129)], nothing), TraceCalls.Trace(logplus10, ([-0.5],), (), TraceCalls.Trace[], 2.25129), TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-2.5], [0.105263], :central), (), TraceCalls.Trace[TraceCalls.Trace(logplus10, ([-2.49998],), (), TraceCalls.Trace[], 2.01491), TraceCalls.Trace(logplus10, ([-2.50002],), (), TraceCalls.Trace[], 2.0149)], nothing), TraceCalls.Trace(logplus10, ([-2.5],), (), TraceCalls.Trace[], 2.0149), TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-12.5], [0.133333], :central), (), TraceCalls.Trace[TraceCalls.Trace(logplus10, ([-12.4999],), (), TraceCalls.Trace[], DomainError())], DomainError())], DomainError())], DomainError())

Focusing on just `finite_difference!` shows what happened: the line search keeps going for lower and lower values of `x`, until it tries `log(-12.5+10) == log(-2.5)`, which is a `DomainError`.

In [22]:
filter(tr->tr.func==Calculus.finite_difference!, linesearch_trace)

TraceCalls.Trace(#39, (), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-0.1], [0.133333], :central), (), TraceCalls.Trace[], nothing), TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-0.5], [0.10101], :central), (), TraceCalls.Trace[], nothing), TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-2.5], [0.105263], :central), (), TraceCalls.Trace[], nothing), TraceCalls.Trace(Calculus.finite_difference!, (logplus10, [-12.5], [0.133333], :central), (), TraceCalls.Trace[], DomainError())], DomainError())

`filter` is useful even when there is no exception to debug. By selecting only certain parts of the trace, it's the TraceCalls equivalent of setting a breakpoint. 

As a convenience, `which(::Trace)` and `edit(::Trace)` are implemented. They point to the source location for that function call.

#### Debugging test failures

When a test used to pass, but now fails after a code change, `compare_past_trace` can be used to better understand the failure.

In [23]:
?compare_past_trace

search: compare_past_trace



`compare_past_trace(old_trace::Trace; filter_out_equal=true))` reruns every function call in `old_trace`, and shows in red where the new result differs from the old.  If `filter_out_equal==true`, only show the non-equal results. 


To use it:

1. Trace the failing test/computation under the correct code (it won't fail, of course)
2. `git checkout` the bad code (or make the changes manually). `TraceCalls` imports `Revise.jl`, so the code changes will take effect automatically.
3. Call `compare_past_trace` on the trace from step 1

# Profiling

`measure(:@measuring_macro, trace)` applies the given _quoted_ macro (put a `:` in front of the `@`) to every function call in `trace`. Useful macros are `@allocated`, `@elapsed` and [BenchmarkTools](https://github.com/JuliaCI/BenchmarkTools.jl)' `@belapsed`. Because measuring involves rerunning every function call in the trace, a good rule-of-thumb is to use the more accurate `@belapsed` when your code to profile takes less than a second, and `@elapsed` when it takes less than a minute.

Remember that due to timing fluctuations, `f(x) = g(x) + ...` does not imply that `@elapsed(f(x)) >= @elapsed(g(x))`

In [24]:
## Profile the PyCall code for accessing Python's `math.pi` 
using PyCall, TraceCalls

math = pyimport(:math)
trace_pycall = @trace PyCall math[:pi];

using BenchmarkTools: @belapsed
measure(:@belapsed, trace_pycall; normalize=true, threshold=0.005)  # only show calls that take >0.5% of total runtime

TraceCalls.Trace(#43, (), (), TraceCalls.Trace[TraceCalls.Trace(getindex, (PyObject <module 'math' from '/Users/cedric/.julia/v0.6/Conda/deps/usr/lib/python2.7/lib-dynload/math.so'>, :pi), (), TraceCalls.Trace[TraceCalls.Trace(getindex, (PyObject <module 'math' from '/Users/cedric/.julia/v0.6/Conda/deps/usr/lib/python2.7/lib-dynload/math.so'>, "pi"), (), TraceCalls.Trace[], TraceCalls.FontColor("1ee100", 0.1164)), TraceCalls.Trace(convert, (PyCall.PyAny, PyObject 3.141592653589793), (), TraceCalls.Trace[TraceCalls.Trace(PyCall.pytype_query, (PyObject 3.141592653589793, PyCall.PyObject), (), TraceCalls.Trace[TraceCalls.Trace(PyCall.pyint_query, (PyObject 3.141592653589793,), (), TraceCalls.Trace[TraceCalls.Trace(PyCall.pyisinstance, (PyObject 3.141592653589793, Ptr{PyCall.PyObject_struct} @0x00000001268e4c98), (), TraceCalls.Trace[], TraceCalls.FontColor("11ee00", 0.0675)), TraceCalls.Trace(PyCall.pyisinstance, (PyObject 3.141592653589793, Ptr{PyCall.PyObject_struct} @0x00000001268e6aa8), (), TraceCalls.Trace[], TraceCalls.FontColor("11ee00", 0.0678))], TraceCalls.FontColor("23dc00", 0.138))], TraceCalls.FontColor("7b8400", 0.4843)), TraceCalls.Trace(convert, (Float64, PyObject 3.141592653589793), (), TraceCalls.Trace[TraceCalls.Trace(PyCall.asscalar, (PyObject 3.141592653589793,), (), TraceCalls.Trace[], TraceCalls.FontColor("02fd00", 0.0059)), TraceCalls.Trace(PyCall.pyerr_check, ("ccall(@pysym(:PyFloat_AsDouble), Cdouble, (PyPtr,), asscalar(po))", 3.14159), (), TraceCalls.Trace[], TraceCalls.FontColor("01fe00", 0.0051))], TraceCalls.FontColor("03fc00", 0.0102))], TraceCalls.FontColor("b84700", 0.7235))], TraceCalls.FontColor("f90600", 0.9761))], TraceCalls.FontColor("ff0000", 1.0))